# 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기

## 컬럼 정보
- ID: ID Number of Customers.
- Warehouse block: The Company have big Warehouse which is divided in to block such as A,B,C,D,E.
- Mode of shipment:The Company Ships the products in multiple way such as Ship, Flight and Road.
- Customer care calls: The number of calls made from enquiry for enquiry of the shipment.
- Customer rating: The company has rated from every customer. 1 is the lowest (Worst), 5 is the highest (Best).
- Cost of the product: Cost of the Product in US Dollars.
- Prior purchases: The Number of Prior Purchase.
- Product importance: The company has categorized the product in the various parameter such as low, medium, high.
- Gender: Male and Female.
- Discount offered: Discount offered on that specific product.
- Weight in gms: It is the weight in grams.
- Reached on time: It is the target variable, where 1 Indicates that the product has NOT reached on time and 0  indicates it has reached on time.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# ../ 상위 디렉토리, ./현재 디렉토리

df = pd.read_csv('./Dataset/Train.csv')

In [271]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [272]:
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [273]:
df.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729,0.596691
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251,0.490584
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000,0.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000,0.000000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000,1.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000,1.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000,1.000000


In [274]:
# 결측치가 없응
df.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

In [275]:
# 제품 도착시간 준수 여부
df['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    6563
0    4436
Name: count, dtype: int64

In [276]:
# 범주형 데이터 확인, Mode_of_Shipment, Product_importance, Gender
np.unique(df['Mode_of_Shipment']), np.unique(df['Product_importance']), np.unique(df['Gender'])

(array(['Flight', 'Road', 'Ship'], dtype=object),
 array(['high', 'low', 'medium'], dtype=object),
 array(['F', 'M'], dtype=object))

In [277]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cols = ['Mode_of_Shipment', 'Product_importance', 'Warehouse_block']
for col in cols:
    df[col] = le.fit_transform(df[col])


In [279]:
df.drop('Gender', axis =1, inplace =True)

In [281]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Reached.on.Time_Y.N'], axis=1)
y = df['Reached.on.Time_Y.N']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)


In [283]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8799, 10), (2200, 10), (8799,), (2200,))

In [252]:
# 실제로 모델을 학습하기 전에 필요없는 데이터 정리...
X_train = X_train.drop('ID', axis = 1)
cust_id = X_test.pop('ID')

In [312]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(n_estimators= 100, max_depth=5, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)
roc_auc_score(y_test, y_pred_proba[:,1])

0.7565533251825067

In [299]:
xg = XGBClassifier(eval_metric='mlogloss', learning_rate=0.3, n_estimators = 100)
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)
y_pred_proba = xg.predict_proba(X_test)
roc_auc_score(y_test, y_pred_proba[:,1])


0.7306065270642921

In [290]:
np.sum(y_pred != y_test)

765

In [314]:
from sklearn.metrics import accuracy_score

print(f"total error: {np.sum(y_pred != y_test)} from {len(y_pred)} accuracy: {round(accuracy_score(y_test, y_pred), 2)}")

total error: 679 from 2200 accuracy: 0.69


In [313]:
submission = pd.DataFrame(data={'ID': cust_id, 'Reached.on.Time_Y.N': y_pred_proba[:,1]}
                          )
submission.round(3)

,ID,Reached.on.Time_Y.N
7030,7031,0.438
2095,2096,0.999
7168,7169,0.465
7969,7970,0.434
6320,6321,0.435
...,...,...
8986,8987,0.426
10540,10541,0.431
9009,9010,0.419
5212,5213,0.422


In [300]:
submission.to_csv('09_25_13_30.csv', index=False)